## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,60.04,69,98,6.64,overcast clouds
1,1,Vanavara,RU,60.3400,102.2797,27.45,92,37,6.55,scattered clouds
2,2,Vredendal,ZA,-31.6683,18.5012,57.20,66,14,13.15,few clouds
3,3,Severo-Kurilsk,RU,50.6789,156.1250,39.27,62,11,13.73,few clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,78.76,85,99,5.99,overcast clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?53
What is the maximum temperature you would like for your trip?69


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)




,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,60.04,69,98,6.64,overcast clouds
2,2,Vredendal,ZA,-31.6683,18.5012,57.20,66,14,13.15,few clouds
6,6,Chapais,CA,49.7834,-74.8492,55.18,96,100,10.80,overcast clouds
9,9,Chuy,UY,-33.6971,-53.4616,60.37,48,6,11.63,clear sky
25,25,East London,ZA,-33.0153,27.9116,61.70,61,6,28.03,clear sky
30,30,Richards Bay,ZA,-28.7830,32.0377,67.86,79,99,15.12,light rain
32,32,Bredasdorp,ZA,-34.5322,20.0403,54.28,52,91,13.85,overcast clouds
41,41,Oranjemund,NaN,-28.5500,16.4333,57.15,67,20,22.12,few clouds
58,58,Lebu,CL,-37.6167,-73.6500,55.31,69,66,12.21,moderate rain
59,59,Jamestown,US,42.0970,-79.2353,57.09,95,90,8.99,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,60.04,overcast clouds,-33.6500,115.3333,
2,Vredendal,ZA,57.20,few clouds,-31.6683,18.5012,
6,Chapais,CA,55.18,overcast clouds,49.7834,-74.8492,
9,Chuy,UY,60.37,clear sky,-33.6971,-53.4616,
25,East London,ZA,61.70,clear sky,-33.0153,27.9116,
30,Richards Bay,ZA,67.86,light rain,-28.7830,32.0377,
32,Bredasdorp,ZA,54.28,overcast clouds,-34.5322,20.0403,
58,Lebu,CL,55.31,moderate rain,-37.6167,-73.6500,
59,Jamestown,US,57.09,overcast clouds,42.0970,-79.2353,
64,Williston,US,57.61,clear sky,48.1470,-103.6180,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,60.04,overcast clouds,-33.6500,115.3333,Observatory Guest House
2,Vredendal,ZA,57.20,few clouds,-31.6683,18.5012,Tharrakamma Guest House
6,Chapais,CA,55.18,overcast clouds,49.7834,-74.8492,Hôtel Opémiska
9,Chuy,UY,60.37,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
25,East London,ZA,61.70,clear sky,-33.0153,27.9116,Tu Casa


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [33]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))